In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from subprocess import call

In [2]:
from directory import directory
directory = "/scratch/shared/mrozemul/Fiji.app/"

plate = 3
dates_datetime = get_dates_datetime(directory,plate)
begin = 0
end = 19
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])

begin = 2020-08-14 00:57:00 
  end = 2020-08-17 04:57:00


In [3]:
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'/scratch/shared/mrozemul/Fiji.app/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [10]:
branch_frequ = []
for hyph in exp.hyphaes:
    if len(hyph.ts)>0:
        t = hyph.ts[-1]
        length = hyph.get_length_um(t)
        nodes = hyph.get_nodes_within(t)
        branch_frequ.append((len(nodes)-1)/(length+1))
        

In [14]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(branch_frequ,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([4.703e+03, 5.390e+02, 1.040e+02, 5.400e+01, 2.400e+01, 1.000e+00,
        1.600e+01, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 2.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.500e+01]),
 array([4.97873260e-05, 3.33814611e-02, 6.67131348e-02, 1.00044809e-01,
        1.33376482e-01, 1.66708156e-01, 2.00039830e-01, 2.33371504e-01,
        2.66703177e-01, 3.00034851e-01, 3.33366525e-01, 3.66698199e-01,
        4.00029872e-01, 4.33361546e-01, 4.66693220e-01, 5.00024894e-01,
        5.33356567e-01, 5.66688241e-01, 6.00019915e-01, 6.33351589e-01,
        6.66683262e-01, 7.00014936e-01, 7.33346610e-01, 7.66678284e-01,
        8.00009957e-01, 8.33341631e-01, 8.66673305e-01, 9.00004979e-01,
        9.33336652e-01, 9.66668326e-01, 1.00000000e+00]),
 <a list of 30 Patch objects>)

In [15]:
np.mean(branch_frequ)

0.021061224646895957